In [1]:
import pandas as pd
import numpy as np
import os

In [6]:
df_hospital = pd.read_csv(os.getcwd() + '/hospital_billing.csv')

In [7]:
df_hospital

,Unnamed: 0,case_id,activity,lifecycle,resource,timestamp,actorange,actred,blocked,casetype,...,iscancelled,isclosed,msgcode,msgcount,msgtype,speciality,state,version,activity_instance_id,.order
0,1,A,NEW,complete,ResA,2012-12-16 19:33:10,NaN,NaN,False,A,...,False,True,NaN,NaN,NaN,A,In progress,NaN,1,1
1,2,A,FIN,complete,NaN,2013-12-15 19:00:37,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,2,2
2,3,A,RELEASE,complete,NaN,2013-12-16 03:53:38,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Released,NaN,3,3
3,4,A,CODE OK,complete,NaN,2013-12-17 12:56:29,False,False,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,A,4,4
4,5,A,BILLED,complete,ResB,2013-12-19 03:44:31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Billed,NaN,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49946,49947,PTN,NEW,complete,ResA,2013-03-27 22:29:13,NaN,NaN,False,A,...,False,True,NaN,NaN,NaN,L,In progress,NaN,49947,49947
49947,49948,PTN,FIN,complete,NaN,2013-05-27 00:43:02,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,49948,49948
49948,49949,PTN,RELEASE,complete,NaN,2013-05-27 01:56:12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Released,NaN,49949,49949
49949,49950,PTN,CODE OK,complete,NaN,2013-05-30 01:42:39,False,False,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,B,49950,49950


In [8]:
df_hospital = df_hospital.rename(columns={"case_id": "caseid", "timestamp": "ts"})

In [9]:
event_log = df_hospital[["caseid","activity","ts"]]

In [ ]:
# Convert 'ts' to datetime format
event_log['ts'] = pd.to_datetime(event_log['ts'])

# Group by 'caseid' and aggregate events into a list ordered by timestamp
CaseidList = event_log.groupby('caseid').agg({
    'ts': lambda x: x.sort_values().tolist(),
    'activity': lambda x: x.tolist()
}).reset_index()

In [11]:
# Calculate duration and event_number
CaseidList['duration'] = CaseidList['ts'].apply(lambda x: max(x) - min(x))
CaseidList['event_number'] = CaseidList['activity'].apply(len)

In [12]:
CaseidList

,caseid,ts,activity,duration,event_number
0,A,"[2012-12-16 19:33:10, 2013-12-15 19:00:37, 201...","[NEW, FIN, RELEASE, CODE OK, BILLED]",367 days 08:11:21,5
1,AA,"[2012-12-26 08:50:18, 2012-12-26 08:50:59, 201...","[NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...",193 days 10:31:09,6
2,AAA,[2013-01-26 22:56:11],[NEW],0 days 00:00:00,1
3,AAB,[2013-01-03 22:55:39],[NEW],0 days 00:00:00,1
4,AAC,"[2013-01-08 11:37:19, 2013-01-08 11:37:37, 201...","[NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...",186 days 12:20:43,6
...,...,...,...,...,...
9994,ZZI,"[2013-02-25 19:35:08, 2013-04-11 19:42:23, 201...","[NEW, FIN, RELEASE, CODE OK, REOPEN, DELETE]",117 days 18:41:22,6
9995,ZZJ,[2013-02-27 22:59:03],[NEW],0 days 00:00:00,1
9996,ZZK,"[2013-03-11 10:38:05, 2013-03-11 10:38:26, 201...","[NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...",116 days 11:07:50,6
9997,ZZL,"[2013-04-06 02:38:48, 2013-04-11 03:37:44, 201...","[NEW, FIN, RELEASE, CODE OK, BILLED]",158 days 07:16:47,5


In [13]:
# Find the longest case by duration
longest_case = CaseidList.loc[CaseidList['duration'].idxmax()]

# Find the shortest case by duration
shortest_case = CaseidList.loc[CaseidList['duration'].idxmin()]

In [14]:
longest_case

caseid                                                        GTC
ts              [2013-01-13 03:47:04, 2014-01-14 05:26:29, 201...
activity            [NEW, FIN, RELEASE, CODE OK, BILLED, CODE OK]
duration                                       1035 days 10:06:32
event_number                                                    6
Name: 2599, dtype: object

In [15]:
shortest_case

caseid                            AAA
ts              [2013-01-26 22:56:11]
activity                        [NEW]
duration              0 days 00:00:00
event_number                        1
Name: 2, dtype: object

In [16]:
longest_case_by_nof_events = CaseidList[CaseidList['event_number'] == CaseidList['event_number'].max()]

In [17]:
longest_case_by_nof_events

,caseid,ts,activity,duration,event_number
4648,MBL,"[2013-04-04 21:12:12, 2013-04-04 21:12:13, 201...","[NEW, NEW, NEW, NEW, NEW, NEW, NEW, NEW, NEW, ...",375 days 01:52:29,217
